# Generate Dataframe from RDAT

This notebook takes an input RDAT file with reactivity data for a given sequence library,
and extracts the relevant data and stores it in a Dataframe.

In [ ]:
import sys
sys.path.append('..')

import os
import pandas as pd
from openknotscore.utils import load_rdat, get_global_blank_out

# Trick to display dataframes with scrollbars
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

In [ ]:
# Generate empty dataframe with columns that we will extract from the RDAT

df = pd.DataFrame(columns=["id","title","author","sequence","reads","signal_to_noise","snr","warning","reactivity","errors"])

for file in os.listdir('../data/rdats'):
    if not file.endswith('.rdat'):
        continue
    rdat, constructName = load_rdat(f'../data/rdats/{file}')

    seqList = []

    BLANK_OUT5, BLANK_OUT3 = get_global_blank_out(rdat.constructs[constructName])

    # Loop through all sequences in the RDAT, extract relevant data, and add to the dataframe
    for sequence in rdat.constructs[constructName].data:
        # Grab annotations
        id = None
        author = None
        title = None
        score_start_idx = None
        score_end_idx = None
        if 'Eterna' in sequence.annotations:
            for annot in sequence.annotations.get('Eterna'):
                if annot.startswith('id:'):
                    id = annot.removeprefix('id:')
                if annot.startswith('author:'):
                    author = annot.removeprefix('author:')
                if annot.startswith('design_name:'):
                    title = annot.removeprefix('design_name:')
                if annot.startswith('score:start_idx:'):
                    score_start_idx = int(annot.removeprefix('score:start_idx:'))
                if annot.startswith('score:end_idx:'):
                    score_end_idx = int(annot.removeprefix('score:end_idx:'))
        if title is None:
            title = sequence.annotations.get('name')[0]
        if id is None:
            id = f'EXTERNAL:{title}'
        seq = sequence.annotations.get('sequence')[0]
        reads = None
        if 'reads' in sequence.annotations:
            reads = int(sequence.annotations.get('reads')[0])
        signal_to_noise = sequence.annotations.get('signal_to_noise')[0]
        snr = float(sequence.annotations.get('signal_to_noise')[0].split(":")[1])
        warning = sequence.annotations.get('warning', '-')[0]
        if score_start_idx is None:
            score_start_idx = BLANK_OUT5
        if score_end_idx is None:
            score_end_idx = BLANK_OUT3

        # Get reactivity data and errors
        reactivity = [float('nan')]*BLANK_OUT5 + sequence.values + [float('nan')]*BLANK_OUT3
        errors = [float('nan')]*BLANK_OUT5 + sequence.errors + [float('nan')]*BLANK_OUT3

        # Create a dataframe from this row
        row = pd.DataFrame(
            data = [[id,seq,reads,signal_to_noise,snr,warning,reactivity,errors,score_start_idx,score_end_idx]],
            columns=["id","sequence","reads","signal_to_noise","snr","warning","reactivity","errors","score_start_idx","score_end_idx"]
        )

        # Add the row to the whole dataframe
        seqList.append(row)

df = pd.concat(seqList, ignore_index=True)

In [ ]:
# Reset the index and sort by ID
df.set_index('id', inplace=True)
df.sort_values('id',ascending=False,inplace=True)

# Persist the dataframe for loading in future steps
# See the README for details about format choice

# df.to_csv('../data/data_rdatOnly.csv')
df.to_pickle('../data/data_rdatOnly.pkl')

In [ ]:
# OPTIONAL
# Some sequences will have poor data quality, indicated with a badQuality warning
# To save compute, you may choose to filter out the low quality sequences

goodQuality = df.loc[df['warning'] != 'badQuality']

# goodQuality.to_csv('../data/data_highQuality.csv')
goodQuality.to_pickle('../data/data_highQuality.pkl')